In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import tensorflow as tf
import os
import numpy as np
import tensorflow_io as tfio

from helper_functions import (
    create_spectrogram_features,
    lite_model_from_file_predicts_dataset,
    get_file_size, 
    convert_bytes,
    evaluate_prediction,
)


from sklearn.metrics import confusion_matrix

In [6]:
desired_length_of_audio = 48000
sample_rate = 16000

In [13]:
# Take all audio from testing dataset and create spectrograms from them
# We will use spectrograms for models testing
directory = 'dataset/testing'

x_data = []
y_data = []
for root, dirs, files in os.walk(directory):
    for file in files:
        full_file_name = os.path.join(root, file)

        if "non_target" in str(full_file_name):
            class_encoded = 0
        elif "target" in str(full_file_name):
            class_encoded = 1

        audio, sr = tf.audio.decode_wav(tf.io.read_file(full_file_name))
        audio = tf.squeeze(audio, axis=-1)
        resampled_audio = tfio.audio.resample(audio, rate_in=48000, rate_out=sample_rate)
        # Prepare log mel spectrogram from audio
        spectrogram_feature = create_spectrogram_features(resampled_audio, desired_length=48000, sample_rate = 16000)
        x_data.append(spectrogram_feature)
        y_data.append(class_encoded)

# input data should be in numpy array, not in list
x_data_np = np.array(x_data)
y_data_np = np.array(y_data)

In [14]:
for i in x_data_np:
    print(i.shape)
    break

(184, 80, 1)


### CNN

##### CNN initial model in keras format

In [15]:
cnn_initial_model_path = 'spectrogram_models_to_test_with_RIOT_ML/cnn/cnn_mel_spec_16kHz_baseline.keras'
convert_bytes(get_file_size(cnn_initial_model_path), "KB")

File size: 335.824 Kilobytes


In [16]:
cnn_initial_model = tf.keras.models.load_model(cnn_initial_model_path)
# input data should be in numpy array
y_pred_prob = cnn_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model (without any additional quantization techniques)

In [19]:
cnn_tflite_model_path = 'spectrogram_models_to_test_with_RIOT_ML/cnn/cnn_mel_spec_16kHz.tflite'
convert_bytes(get_file_size(cnn_tflite_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_tflite_model_path, x_data_np, y_data_np)

File size: 103.684 Kilobytes
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model + Post Training Dynamic range quantization

In [20]:
cnn_drq_model_path = 'spectrogram_models_to_test_with_RIOT_ML/cnn/cnn_mel_spec_16kHz_drq.tflite'
convert_bytes(get_file_size(cnn_drq_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_drq_model_path, x_data_np, y_data_np)

File size: 29.508 Kilobytes
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model + Float 16 quantization

In [29]:
cnn_float16q_model_path = 'spectrogram_models_to_test_with_RIOT_ML/cnn/cnn_mel_spec_16KHz_float16q.tflite'
convert_bytes(get_file_size(cnn_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_float16q_model_path, x_data_np, y_data_np)

File size: 54.879 Kilobytes
Accuracy: 98.99%
Recall: 99.13%
Precision: 97.84%
F1-score: 98.48%


##### CNN tf Lite model + Full integer quantization

In [24]:
cnn_full_int_q_model_path = 'spectrogram_models_to_test_with_RIOT_ML/cnn/cnn_mel_spec_16kHz_full_int_q.tflite'
convert_bytes(get_file_size(cnn_full_int_q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(cnn_full_int_q_model_path, x_data_np, y_data_np, input_data_uint8_type=True)

File size: 29.898 Kilobytes
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### SqueezeNet

##### SqueezeNet initial model in keras format

In [30]:
squeezenet_initial_model_path = 'spectrogram_models_to_test_with_RIOT_ML/squeezenet/squeezenet_spec_16kHz_baseline.keras'
convert_bytes(get_file_size(squeezenet_initial_model_path), "KB")

File size: 8687.448 Kilobytes


In [31]:
squeezenet_initial_model = tf.keras.models.load_model(squeezenet_initial_model_path)
# input data should be in numpy array
y_pred_prob = squeezenet_initial_model.predict(x_data_np, verbose=0)
y_pred = tf.argmax(y_pred_prob, axis=1).numpy()

# Evaluate
evaluate_prediction(y_data_np, y_pred)

Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### SqueezeNet tf Lite model (without any additional quantization techniques)

In [32]:
squeezenet_tflite_model_path = 'spectrogram_models_to_test_with_RIOT_ML/squeezenet/squeezenet_spec_16kHz.tflite'
convert_bytes(get_file_size(squeezenet_tflite_model_path), "KB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_model_path, x_data_np, y_data_np)

File size: 2856.07 Kilobytes
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### SqueezeNet tf Lite model + Post Training Dynamic range quantization

In [33]:
squeezenet_tflite_drq_model_path = 'spectrogram_models_to_test_with_RIOT_ML/squeezenet/squeezenet_spec_16kHz_drq.tflite'
convert_bytes(get_file_size(squeezenet_tflite_drq_model_path), "KB")
y_pred = lite_model_from_file_predicts_dataset(squeezenet_tflite_drq_model_path, x_data_np, y_data_np)

File size: 770.258 Kilobytes
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### SqueezeNet tf Lite model + Float 16 quantization

In [34]:
squeezenet_float16q_model_path = 'spectrogram_models_to_test_with_RIOT_ML/squeezenet/squeezenet_spec_16kHz_float16q.tflite'
convert_bytes(get_file_size(squeezenet_float16q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(squeezenet_float16q_model_path, x_data_np, y_data_np)

File size: 1442.977 Kilobytes
Accuracy: 98.56%
Recall: 99.56%
Precision: 96.20%
F1-score: 97.85%


##### CNN tf Lite model + Full integer quantization

In [35]:
squeezenet_full_int_q_model_path = 'spectrogram_models_to_test_with_RIOT_ML/squeezenet/squeezenet_spec_16kHz_full_int_q.tflite'
convert_bytes(get_file_size(squeezenet_full_int_q_model_path), "KB")

y_pred = lite_model_from_file_predicts_dataset(squeezenet_full_int_q_model_path, x_data_np, y_data_np, input_data_uint8_type=True)

File size: 807.539 Kilobytes
Accuracy: 67.12%
Recall: 0.00%
Precision: 0.00%
F1-score: 0.00%


/home/polina/test/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
